# Nelson Siegel Svensson (NSS) Interest Rate Model

A more advanced model to simulate interest rates. 

In [5]:
import scipy.optimize as optimize
from src.data import dataset
import matplotlib.pyplot as plt
import numpy as np

In [4]:
zerocurve = dataset.Zerocurve()
zerocurve.load_data()
df_z = zerocurve.df
start_date = df_z.index.min()
end_date = df_z.index.max()
yield_data = df_z.pivot(columns = 'tenor', values = 'rate')

In [ ]:
# yield columns must be in year fractions

In [7]:
# Define the NSS function
def nss(t, beta0, beta1, beta2, beta3, tau1, tau2):
    return beta0 + beta1 * (1 - np.exp(-t/tau1))/(t/tau1) + beta2 * ((1 - np.exp(-t/tau1))/(t/tau1) - np.exp(-t/tau1)) + beta3 * ((1 - np.exp(-t/tau2))/(t/tau2) - np.exp(-t/tau2))

# Define the objective function to minimize the sum of squared errors between the actual and predicted yield curve
def nss_obj(x, t, y):
    return np.sum((nss(t, *x) - y)**2)

# Fit the NSS model to the historical yield data
t = np.array(yield_data.columns.values, dtype=np.float64)
y = yield_data.iloc[-1].values
x0 = [1, -0.01, -0.01, -0.01, 1, 1]
res = optimize.minimize(nss_obj, x0, args=(t, y))
beta0, beta1, beta2, beta3, tau1, tau2 = res.x

# Define a function to simulate the NSS model
def nss_simulate(t, beta0, beta1, beta2, beta3, tau1, tau2):
    return nss(t, beta0, beta1, beta2, beta3, tau1, tau2)

# Define the time horizon and time step
T = 1 # time horizon
dt = 1/252 # time step

# Simulate the NSS model
yield_curve = [nss_simulate(t[-1] + dt, beta0, beta1, beta2, beta3, tau1, tau2)]
for i in range(int(T/dt)):
    t_i = t[-1] + (i+1)*dt
    yield_i = nss_simulate(t_i, beta0, beta1, beta2, beta3, tau1, tau2)
    yield_curve.append(yield_i)

# Plot the simulated yield curve
plt.plot(yield_curve)
plt.show()

ValueError: could not convert string to float: '1 Year'